# AOC2022

## Day 22 / Part 2 / Monkey Map

Problem Description: https://adventofcode.com/2022/day/22#part2

Input: [Example](aoc2022_day22_example.txt)

### Notes
- The cube can be visualized e.g. by printing the board map on paper, then, cutting and folding it
- Depending on the arrangement of the regions on the board map the Index Map must be adapted to correctly map positions from the board map to positions on the cube
  <img src="aoc2022_day22_notes.png" width="400" style="margin: 0px;"/>
- The algorithm consists of four steps:
  1. Extract board map regions from the puzzle input
  2. Build a cube from the regions
  3. Traverse the path from the puzzle input on the cube
  4. Map the found solution on the cube to the board map

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 22, part 2."""
# required for forward references (causing E0602)
from __future__ import annotations

from dataclasses import dataclass
import enum
import itertools
import logging
import re
import sys
import numpy as np

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day22_example.txt"

In [3]:
class Face(enum.IntEnum):
    """Face enumeration."""
    TOP = 0
    LEFT = 1
    FRONT = 2
    RIGHT = 3
    BOTTOM = 4
    BACK = 5


class Direction(enum.IntEnum):
    """Direction enumeration."""
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3


@dataclass
class Node:
    """Class representing a cube node."""
    pos: list[int]
    up_node: Node = None
    right_node: Node = None
    down_node: Node = None
    left_node: Node = None

    def __init__(self, pos):
        self.pos = pos

    def __eq__(self, other):
        return self.pos == other.pos

    def __str__(self):
        return str(self.pos)


class Cube:
    """Cube of linked nodes."""
    sides = None
    nodes = None

    def __init__(self, sides):
        self.side_shape = sides[Face.TOP].shape
        self.sides = sides
        self._build()

    def _compute_up_node_pos(self, face, side, y_pos, x_pos):
        """
        Internal method for computing the upper neighbour of a node.
        """
        up_node_pos = None
        if y_pos-1 < 0:
            if (
                face == Face.TOP and
                self.sides[Face.BACK][self.side_shape[0]-1, x_pos] != "#"
            ):
                up_node_pos = (Face.BACK, self.side_shape[0]-1, x_pos)
            if (
                face == Face.LEFT and
                self.sides[Face.TOP][x_pos, 0] != "#"
            ):
                up_node_pos = (Face.TOP, x_pos, 0)
            if (
                face == Face.FRONT and
                self.sides[Face.TOP][self.side_shape[0]-1, x_pos] != "#"
            ):
                up_node_pos = (Face.TOP, self.side_shape[0]-1, x_pos)
            if (
                face == Face.RIGHT and
                self.sides[Face.TOP][
                    self.side_shape[0]-1-x_pos, self.side_shape[1]-1
                ] != "#"
            ):
                up_node_pos = (
                    Face.TOP, self.side_shape[0]-1-x_pos, self.side_shape[1]-1
                )
            if (
                face == Face.BOTTOM and
                self.sides[Face.FRONT][self.side_shape[0]-1, x_pos] != "#"
            ):
                up_node_pos = (Face.FRONT, self.side_shape[0]-1, x_pos)
            if (
                face == Face.BACK and
                self.sides[Face.BOTTOM][self.side_shape[0]-1, x_pos] != "#"
            ):
                up_node_pos = (Face.BOTTOM, self.side_shape[0]-1, x_pos)
        else:
            if side[y_pos-1, x_pos] != "#":
                up_node_pos = (face, y_pos-1, x_pos)
        return up_node_pos

    def _compute_right_node_pos(self, face, side, y_pos, x_pos):
        """
        Internal method for computing the right neighbour of a node.
        """
        right_node_pos = None
        if x_pos+1 >= self.side_shape[1]:
            if (
                face == Face.TOP and
                self.sides[Face.RIGHT][0, self.side_shape[1]-1-y_pos] != "#"
            ):
                right_node_pos = (Face.RIGHT, 0, self.side_shape[1]-1-y_pos)
            if (
                face == Face.LEFT and
                self.sides[Face.FRONT][y_pos, 0] != "#"
            ):
                right_node_pos = (Face.FRONT, y_pos, 0)
            if (
                face == Face.FRONT and
                self.sides[Face.RIGHT][y_pos, 0] != "#"
            ):
                right_node_pos = (Face.RIGHT, y_pos, 0)
            if (
                face == Face.RIGHT and
                self.sides[Face.BACK][
                    self.side_shape[0]-1-y_pos, self.side_shape[1]-1
                ] != "#"
            ):
                right_node_pos = (
                    Face.BACK, self.side_shape[0]-1-y_pos, self.side_shape[1]-1
                )
            if (
                face == Face.BOTTOM and
                self.sides[Face.RIGHT][self.side_shape[0]-1, y_pos] != "#"
            ):
                right_node_pos = (Face.RIGHT, self.side_shape[0]-1, y_pos)
            if (
                face == Face.BACK and
                self.sides[Face.RIGHT][
                    self.side_shape[0]-1-y_pos, self.side_shape[1]-1
                ] != "#"
            ):
                right_node_pos = (
                    Face.RIGHT,
                    self.side_shape[0]-1-y_pos, self.side_shape[1]-1
                )
        else:
            if side[y_pos, x_pos+1] != "#":
                right_node_pos = (face, y_pos, x_pos+1)
        return right_node_pos

    def _compute_down_node_pos(self, face, side, y_pos, x_pos):
        """
        Internal method for computing the below neighbour of a node.
        """
        down_node_pos = None
        if y_pos+1 >= self.side_shape[0]:
            if (
                face == Face.TOP and
                self.sides[Face.FRONT][0, x_pos] != "#"
            ):
                down_node_pos = (Face.FRONT, 0, x_pos)
            if (
                face == Face.LEFT and
                self.sides[Face.BOTTOM][self.side_shape[0]-1-x_pos, 0] != "#"
            ):
                down_node_pos = (Face.BOTTOM, self.side_shape[0]-1-x_pos, 0)
            if (
                face == Face.FRONT and
                self.sides[Face.BOTTOM][0, x_pos] != "#"
            ):
                down_node_pos = (Face.BOTTOM, 0, x_pos)
            if (
                face == Face.RIGHT and
                self.sides[Face.BOTTOM][x_pos, self.side_shape[1]-1] != "#"
            ):
                down_node_pos = (Face.BOTTOM, x_pos, self.side_shape[1]-1)
            if (
                face == Face.BOTTOM and
                self.sides[Face.BACK][0, x_pos] != "#"
            ):
                down_node_pos = (Face.BACK, 0, x_pos)
            if (
                face == Face.BACK and
                self.sides[Face.TOP][0, x_pos] != "#"
            ):
                down_node_pos = (Face.TOP, 0, x_pos)
        else:
            if side[y_pos+1, x_pos] != "#":
                down_node_pos = (face, y_pos+1, x_pos)
        return down_node_pos

    def _compute_left_node_pos(self, face, side, y_pos, x_pos):
        """
        Internal method for computing the left neighbour of a node.
        """
        left_node_pos = None
        if x_pos-1 < 0:
            if (
                face == Face.TOP and
                self.sides[Face.LEFT][0, y_pos] != "#"
            ):
                left_node_pos = (Face.LEFT, 0, y_pos)
            if (
                face == Face.LEFT and
                self.sides[Face.BACK][self.side_shape[0]-1-y_pos, 0] != "#"
            ):
                left_node_pos = (Face.BACK, self.side_shape[0]-1-y_pos, 0)
            if (
                face == Face.FRONT and
                self.sides[Face.LEFT][y_pos, self.side_shape[1]-1] != "#"
            ):
                left_node_pos = (Face.LEFT, y_pos, self.side_shape[1]-1)
            if (
                face == Face.RIGHT and
                self.sides[Face.FRONT][y_pos, self.side_shape[1]-1] != "#"
            ):
                left_node_pos = (Face.FRONT, y_pos, self.side_shape[1]-1)
            if (
                face == Face.BOTTOM and
                self.sides[Face.LEFT][
                    self.side_shape[0]-1, self.side_shape[1]-1-y_pos
                ] != "#"
            ):
                left_node_pos = (
                    Face.LEFT, self.side_shape[0]-1, self.side_shape[1]-1-y_pos
                )
            if (
                face == Face.BACK and
                self.sides[Face.LEFT][self.side_shape[0]-1-y_pos, 0] != "#"
            ):
                left_node_pos = (Face.LEFT, self.side_shape[0]-1-y_pos, 0)
        else:
            if side[y_pos, x_pos-1] != "#":
                left_node_pos = (face, y_pos, x_pos-1)
        return left_node_pos

    def _build(self):
        """Internal method for building the cube."""
        self.nodes = {}
        for face, side in self.sides.items():
            for y_pos, x_pos in (
                (y_pos, x_pos)
                for y_pos in range(side.shape[0])
                for x_pos in range(side.shape[1])
            ):
                if side[y_pos, x_pos] != ".":
                    continue

                node_pos = (face, y_pos, x_pos)
                if node_pos not in self.nodes:
                    self.nodes[node_pos] = Node(node_pos)

                # 1. compute neighbor node positions
                up_node_pos = \
                    self._compute_up_node_pos(face, side, y_pos, x_pos)
                right_node_pos = \
                    self._compute_right_node_pos(face, side, y_pos, x_pos)
                down_node_pos = \
                    self._compute_down_node_pos(face, side, y_pos, x_pos)
                left_node_pos = \
                    self._compute_left_node_pos(face, side, y_pos, x_pos)

                # 2. link nodes
                for direction, neigh_node_pos in enumerate([
                    up_node_pos, right_node_pos,
                    down_node_pos, left_node_pos
                ]):
                    if neigh_node_pos is None:
                        continue
                    neigh_node = None
                    if neigh_node_pos in self.nodes:
                        neigh_node = self.nodes[neigh_node_pos]
                    else:
                        neigh_node = Node(neigh_node_pos)
                        self.nodes[neigh_node_pos] = neigh_node

                    if direction == Direction.UP:
                        self.nodes[node_pos].up_node = neigh_node
                    if direction == Direction.RIGHT:
                        self.nodes[node_pos].right_node = neigh_node
                    if direction == Direction.DOWN:
                        self.nodes[node_pos].down_node = neigh_node
                    if direction == Direction.LEFT:
                        self.nodes[node_pos].left_node = neigh_node

    def __str__(self):
        """Compute the informal string representation of the node."""
        return self.sides

    def __getitem__(self, idx):
        """Get node at index idx."""
        return self.nodes[idx]


class CubeWalker:
    """Helper class for walking on a cube of nodes."""
    curr_node = None
    direction = 0

    def __init__(self, start_node, start_direction=Direction.RIGHT):
        self.curr_node = start_node
        self.direction = start_direction

    def turn_left(self):
        """Change facing direction to the left."""
        self.direction = (self.direction - 1) % 4

    def turn_right(self):
        """Change facing direction to the right."""
        self.direction = (self.direction + 1) % 4

    def move(self, steps):
        """Move forward an amount of steps."""
        for _ in range(steps):
            next_node = None
            if self.direction == Direction.UP:
                next_node = self.curr_node.up_node
            if self.direction == Direction.RIGHT:
                next_node = self.curr_node.right_node
            if self.direction == Direction.DOWN:
                next_node = self.curr_node.down_node
            if self.direction == Direction.LEFT:
                next_node = self.curr_node.left_node

            if next_node is None:
                return

            if self.curr_node.pos[0] == next_node.pos[0]:
                # we stay on the same face
                pass
            else:
                if (
                    next_node.up_node is not None and
                    next_node.up_node == self.curr_node
                ):
                    self.direction = Direction.DOWN
                if (
                    next_node.right_node is not None and
                    next_node.right_node == self.curr_node
                ):
                    self.direction = Direction.LEFT
                if (
                    next_node.down_node is not None and
                    next_node.down_node == self.curr_node
                ):
                    self.direction = Direction.UP
                if (
                    next_node.left_node is not None and
                    next_node.left_node == self.curr_node
                ):
                    self.direction = Direction.RIGHT
            self.curr_node = next_node

In [4]:
INDEX_MAP = {
    # pylint: disable=E1121
    Face.TOP: np.array(
        list(itertools.product(range(0, 4), range(8, 12)))
    ).reshape(4, 4, 2),
    Face.LEFT: np.array(
        list(itertools.product(range(4, 8), range(4, 8)))
    ).reshape(4, 4, 2),
    Face.FRONT: np.array(
        list(itertools.product(range(4, 8), range(8, 12)))
    ).reshape(4, 4, 2),
    Face.RIGHT: np.rot90(np.array(
        list(itertools.product(range(8, 12), range(12, 16)))
    ).reshape(4, 4, 2), 1),
    Face.BOTTOM: np.array(
        list(itertools.product(range(8, 12), range(8, 12)))
    ).reshape(4, 4, 2),
    Face.BACK: np.rot90(np.array(
        list(itertools.product(range(4, 8), range(0, 4)))
    ).reshape(4, 4, 2), 2)
}


def to_cube_sides(board_map):
    """
    Utility function for transforming board map regions into cube sides.

    A board map position (y, x) points to a field within a 2d map of regions.
    A cube position (face, y, x) points to a node on a cube of sides
    (identified by faces).

    The board map region-arrangement is expected to be in the ABCDEF shape:
       A
     FBC
       DE
    Hereby, the regions ABCDEF are mapped to the following cube sides/faces:
    A: top, B: left, C: front, D: right, E: bottom, F: back

    If the regions of a board map do not match the ABCDEF shape the INDEX_MAP
    must be adapted.
    """
    return {
        Face.TOP: board_map[
            INDEX_MAP[Face.TOP][:, :, 0],
            INDEX_MAP[Face.TOP][:, :, 1]
        ],
        Face.LEFT: board_map[
            INDEX_MAP[Face.LEFT][:, :, 0],
            INDEX_MAP[Face.LEFT][:, :, 1]
        ],
        Face.FRONT: board_map[
            INDEX_MAP[Face.FRONT][:, :, 0],
            INDEX_MAP[Face.FRONT][:, :, 1]
        ],
        Face.RIGHT: board_map[
            INDEX_MAP[Face.RIGHT][:, :, 0],
            INDEX_MAP[Face.RIGHT][:, :, 1]
        ],
        Face.BOTTOM: board_map[
            INDEX_MAP[Face.BOTTOM][:, :, 0],
            INDEX_MAP[Face.BOTTOM][:, :, 1]
        ],
        Face.BACK: board_map[
            INDEX_MAP[Face.BACK][:, :, 0],
            INDEX_MAP[Face.BACK][:, :, 1]
        ]
    }


def to_board_map_region_pos(node):
    """
    Utility function for transforming a cube node into a board map position.

    A board map position (y, x) points to a field within a 2d map of regions.
    A cube position (face, y, x) points to a node on a cube of sides
    (identified by faces).

    The board map region-arrangement is expected to be in the ABCDEF shape:
       A
     FBC
       DE
    Hereby, the regions ABCDEF are mapped to the following cube sides/faces:
    A: top, B: left, C: front, D: right, E: bottom, F: back

    If the regions of a board map do not match the ABCDEF shape the INDEX_MAP
    must be adapted.
    """
    face, y_pos, x_pos = node.pos
    return list(INDEX_MAP[face][y_pos, x_pos])


def to_board_map_direction(face, direction):
    """
    Utility function for transforming the direction on a cude side/face
    into the direction on the board map.
    """
    if INDEX_MAP[face][0, 0][0] < INDEX_MAP[face][1, 0][0]:
        pass
    elif INDEX_MAP[face][0, 0][0] > INDEX_MAP[face][1, 0][0]:
        # reverse 180° left rotation
        direction = (direction + 2) % 4
    else:
        if INDEX_MAP[face][0, 0][0] < INDEX_MAP[face][0, 1][0]:
            # reverse 90° left rotation
            direction = (direction + 1) % 4
        else:
            # reverse 90° right rotation
            direction = (direction + 3) % 4
    return direction

In [5]:
def main():
    """Main function to solve puzzle."""
    board_map = []
    path = []
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        read_mode = "map"
        for line in [line.rstrip() for line in file_obj.readlines()]:
            if line == "":
                read_mode = "path"
                continue
            if read_mode == "map":
                board_map.append(list(line))
            if read_mode == "path":
                path = line

    # make all rows the same width
    for row in board_map:
        row.extend(
            list(" " * (max(map(len, board_map)) - len(row)))
        )

    start_pos = (Face.TOP, 0, 0)
    board_map = np.asarray(board_map)

    LOGGER.debug(
        "board map:\n  %s\n",
        "\n  ".join(["".join(row) for row in board_map])
    )

    cube = Cube(to_cube_sides(board_map))
    cube_walker = CubeWalker(cube[start_pos])

    LOGGER.debug("move along cube...")
    LOGGER.debug("  traverse node %s", start_pos)
    for steps, turn in re.findall(r"(\d+)([RL]|)", path):
        cube_walker.move(int(steps))
        if turn == "L":
            cube_walker.turn_left()
        if turn == "R":
            cube_walker.turn_right()
        LOGGER.debug("  traverse node %s", cube_walker.curr_node.pos)

    LOGGER.debug("")

    cube_face, _, _ = cube_walker.curr_node.pos
    map_dir = to_board_map_direction(
        cube_face, cube_walker.direction
    )
    map_y_pos, map_x_pos = to_board_map_region_pos(
        cube_walker.curr_node
    )

    print(
        "solution: " + str(
            (map_y_pos+1) * 1000 + (map_x_pos+1) * 4 +
            ((map_dir-1) % 4)
        )
    )

In [6]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 5031
